# **Training for yolo-v8s**


In [1]:
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/weedzip/unlabeled/agri_0_5225_jpeg.rf.ab0d818fca3b3b0ef3411dea1a56000f.jpg
/kaggle/input/weedzip/unlabeled/agri_0_2390_jpeg.rf.b4af55b63fe3ad51e5ccaa7fae52a15a.jpg
/kaggle/input/weedzip/unlabeled/agri_0_2633_jpeg.rf.80320a4fd05d32f7fae265c2d09ef7c4.jpg
/kaggle/input/weedzip/unlabeled/agri_0_1509_jpeg.rf.0d24077902e5229abad25227f9b58df9.jpg
/kaggle/input/weedzip/unlabeled/agri_0_5384_jpeg.rf.225f34308f10dabf4afb9fbd5bf8cad0.jpg
/kaggle/input/weedzip/unlabeled/agri_0_5871_jpeg.rf.a235335bb1259f7e3cb9516b56507792.jpg
/kaggle/input/weedzip/unlabeled/agri_0_4972_jpeg.rf.0f54a6d5540f2866c50b98c4dbd1ed49.jpg
/kaggle/input/weedzip/unlabeled/agri_0_8620_jpeg.rf.f4396b8c17016387764f952e993f1034.jpg
/kaggle/input/weedzip/unlabeled/agri_0_8405_jpeg.rf.c1df9f246e0e6c7beb01a94641c136cc.jpg
/kaggle/input/weedzip/unlabeled/agri_0_5435_jpeg.rf.dd9f861c0fb52517ad87e5e465151543.jpg
/kaggle/input/weedzip/unlabeled/agri_0_491_jpeg.rf.0a7611978e9c0b66c4384b8618709744.jpg
/kaggle/input/weedzip/

# *Importing Libraries*

In [ ]:
import os
import cv2
import shutil
import random
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import yaml

In [2]:
!pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 40.1 MB/s eta 0:00:00


# *Data Preprocessing*

In [4]:
def create_directory_structure(base_path):
    """Create the necessary directory structure for the dataset."""
    folders = [
        "train/images", "train/labels",
        "val/images", "val/labels",
        "test/images", "test/labels"
    ]
    for folder in folders:
        os.makedirs(os.path.join(base_path, folder), exist_ok=True)
    return folders

def adjust_annotations(annotations, crop_x, crop_y, crop_w, crop_h, orig_w, orig_h):
    """Adjust bounding box annotations after cropping."""
    new_annotations = []
    for line in annotations:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        x_abs, y_abs = x_center * orig_w, y_center * orig_h
        width, height = width * orig_w, height * orig_h
        
        x1, y1 = x_abs - width / 2, y_abs - height / 2
        x2, y2 = x_abs + width / 2, y_abs + height / 2
        
        if x1 >= crop_x and y1 >= crop_y and x2 <= crop_x + crop_w and y2 <= crop_y + crop_h:
            x1_new, y1_new = x1 - crop_x, y1 - crop_y
            x2_new, y2_new = x2 - crop_x, y2 - crop_y
            x_center_new = (x1_new + x2_new) / 2 / crop_w
            y_center_new = (y1_new + y2_new) / 2 / crop_h
            width_new = (x2_new - x1_new) / crop_w
            height_new = (y2_new - y1_new) / crop_h
            new_annotations.append(f"{class_id} {x_center_new:.6f} {y_center_new:.6f} {width_new:.6f} {height_new:.6f}")
    return new_annotations

def random_crop(image, annotations, crop_size=(224, 224)):
    """Randomly crop image and adjust annotations."""
    h, w, _ = image.shape
    crop_h, crop_w = crop_size
    if crop_h > h or crop_w > w:
        return image, annotations
    x_start = random.randint(0, w - crop_w)
    y_start = random.randint(0, h - crop_h)
    cropped_image = image[y_start:y_start + crop_h, x_start:x_start + crop_w]
    new_annotations = adjust_annotations(annotations, x_start, y_start, crop_w, crop_h, w, h)
    return cropped_image, new_annotations

def invert_colors(image):
    """Invert image colors."""
    return cv2.bitwise_not(image)

def apply_gaussian_blur(image, kernel_size=(5, 5)):
    """Apply Gaussian blur to image."""
    return cv2.GaussianBlur(image, kernel_size, 0)

def get_dir_size(path):
    """Get directory size in MB."""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)  # Convert to MB

# Data augmentations done
# ------------------------------------------------------------------------------

def process_training_set(train_orig, source_paths, dest_paths, augment=True):
    """Process training set with optional augmentation."""
    processed_count = 0
    augmented_count = 0
    
    for img_name in tqdm(train_orig, desc="Processing training images"):
        img_path = os.path.join(source_paths['images'], img_name)
        ann_path = os.path.join(source_paths['annotations'], img_name.replace(".jpg", ".txt"))
        
        if not os.path.exists(ann_path):
            continue
        
        image = cv2.imread(img_path)
        with open(ann_path, "r") as f:
            annotations = f.readlines()
        
        # Copy original
        shutil.copy(img_path, os.path.join(dest_paths['images'], img_name))
        shutil.copy(ann_path, os.path.join(dest_paths['labels'], img_name.replace(".jpg", ".txt")))
        processed_count += 1
        
        if augment:
            # Generate augmentations
            augmentations = [
                random_crop(image, annotations),
                (invert_colors(image), annotations),
                (apply_gaussian_blur(image), annotations)
            ]
            
            for i, (aug_img, aug_ann) in enumerate(augmentations):
                aug_img_name = f"aug_{i}_{img_name}"
                aug_ann_name = f"aug_{i}_{img_name.replace('.jpg', '.txt')}"
                cv2.imwrite(os.path.join(dest_paths['images'], aug_img_name), aug_img)
                with open(os.path.join(dest_paths['labels'], aug_ann_name), "w") as f:
                    f.write("\n".join(aug_ann))
                augmented_count += 1
                
    return processed_count, augmented_count

# Data Splitting

def process_dataset_split(image_list, source_paths, dest_paths, split_name=""):
    """Process dataset split (validation or test)."""
    processed_count = 0
    for img_name in tqdm(image_list, desc=f"Processing {split_name} images"):
        img_path = os.path.join(source_paths['images'], img_name)
        ann_path = os.path.join(source_paths['annotations'], img_name.replace(".jpg", ".txt"))
        
        if not os.path.exists(ann_path):
            continue
        
        shutil.copy(img_path, os.path.join(dest_paths['images'], img_name))
        shutil.copy(ann_path, os.path.join(dest_paths['labels'], img_name.replace(".jpg", ".txt")))
        processed_count += 1
    
    return processed_count

#-------------------------------------------------------------------------------------------------------

def main():
    base_path = "/kaggle/input/weedzip"
    source_paths = {
        'labeled_images': os.path.join(base_path, "labeled/images"),
        'labeled_annotations': os.path.join(base_path, "labeled/annotations"),
        'test_images': os.path.join(base_path, "test/images"),
        'test_annotations': os.path.join(base_path, "test/annotations")
    }
    preprocessed_path = "/kaggle/working/Preprocessed_img_weed"
    folders = create_directory_structure(preprocessed_path)
    all_original_images = os.listdir(source_paths['labeled_images'])
    train_orig, val_orig = train_test_split(all_original_images, test_size=0.2, random_state=42)
    
    # Print initial dataset statistics
    print("\nOriginal dataset statistics:")
    print(f"Total original images: {len(all_original_images)}")
    print(f"Training images before augmentation: {len(train_orig)}")
    print(f"Validation images: {len(val_orig)}")
    
    # Processing dataset
    train_paths = {
        'images': os.path.join(preprocessed_path, "train/images"),
        'labels': os.path.join(preprocessed_path, "train/labels")
    }
    processed_train, augmented_count = process_training_set(
        train_orig,
        {'images': source_paths['labeled_images'], 'annotations': source_paths['labeled_annotations']},
        train_paths
    )
    val_paths = {
        'images': os.path.join(preprocessed_path, "val/images"),
        'labels': os.path.join(preprocessed_path, "val/labels")
    }
    processed_val = process_dataset_split(
        val_orig,
        {'images': source_paths['labeled_images'], 'annotations': source_paths['labeled_annotations']},
        val_paths,
        "validation"
    )
    test_paths = {
        'images': os.path.join(preprocessed_path, "test/images"),
        'labels': os.path.join(preprocessed_path, "test/labels")
    }
    processed_test = process_dataset_split(
        os.listdir(source_paths['test_images']),
        {'images': source_paths['test_images'], 'annotations': source_paths['test_annotations']},
        test_paths,
        "test"
    )
    
    # Print dataset statistics
    print("\nFinal dataset statistics:")
    print("\nImage counts:")
    print(f"Training images (original): {processed_train}")
    print(f"Training images (augmented): {augmented_count}")
    print(f"Training images (total): {processed_train + augmented_count}")
    print(f"Validation images: {processed_val}")
    print(f"Test images: {processed_test}")
    
    print("\nDataset sizes:")
    print(f"Training set size: {get_dir_size(os.path.join(preprocessed_path, 'train')):.2f} MB")
    print(f"Validation set size: {get_dir_size(os.path.join(preprocessed_path, 'val')):.2f} MB")
    print(f"Test set size: {get_dir_size(os.path.join(preprocessed_path, 'test')):.2f} MB")
    print(f"Total dataset size: {get_dir_size(preprocessed_path):.2f} MB")
    
    print("\nFiles per directory:")
    for folder in folders:
        path = os.path.join(preprocessed_path, folder)
        print(f"{folder}: {len(os.listdir(path))} files")

if __name__ == "__main__":
    main()


Original dataset statistics:
Total original images: 200
Training images before augmentation: 160
Validation images: 40


Processing validation images: 100%|██████████| 40/40 [00:00<00:00, 233.86it/s]


Final dataset statistics:

Image counts:
Training images (original): 160
Training images (augmented): 480
Training images (total): 640
Validation images: 40

Dataset sizes:
Training set size: 46.41 MB
Validation set size: 2.52 MB
Total dataset size: 52.01 MB

Files per directory:
train/images: 640 files
train/labels: 640 files
val/images: 40 files
val/labels: 40 files


# *Training Pipeline*

In [11]:
dataset_path = r'/kaggle/working/Preprocessed_img_weed'  
train_path = f"{dataset_path}/train/images"
val_path = f"{dataset_path}/val/images"
test_path = f"{dataset_path}/test/images"

# Defining dataset configuration for yaml file
dataset_config = {
    "path": "/kaggle/working/Preprocessed_img_weed",  
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": 2,  
    "names": ["weed", "crop"] 
}
yaml_path = "/kaggle/working/weed_dataset.yaml"
with open(yaml_path, "w") as yaml_file:
    yaml.dump(dataset_config, yaml_file, default_flow_style=False)

print(f"YAML file created at: {yaml_path}")

model = YOLO("yolov8s.pt")  
model.train(data="/kaggle/working/weed_dataset.yaml", epochs=100, batch=16, imgsz=640, device="cuda")


YAML file created at: /kaggle/working/weed_dataset.yaml


100%|██████████| 21.5M/21.5M [00:00<00:00, 188MB/s]


Ultralytics 8.3.71 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/weed_dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

100%|██████████| 755k/755k [00:00<00:00, 14.2MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.82it/s]


                   all        207        262      0.909       0.97      0.977      0.913
                  weed         68        131      0.958      0.992      0.985      0.949
            background         91        131      0.861      0.947      0.968      0.878


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 4.6ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c6f7c617b20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [12]:
model = YOLO("/kaggle/working/runs/detect/train/weights/best.pt")  
# Evaluating on test dataset
metrics = model.val(data="/kaggle/working/weed_dataset.yaml", split="test")

Ultralytics 8.3.71 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /kaggle/working/Preprocessed_img_weed/test/labels... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<00:00, 1155.76it/s]

val: New cache created: /kaggle/working/Preprocessed_img_weed/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.84it/s]


                   all         50         74       0.96      0.785      0.878      0.574
                  weed         24         42          1      0.853      0.913      0.633
            background         26         32       0.92      0.716      0.844      0.516


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 8.9ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/val
